In [1]:
from fastai.vision.all import *
import albumentations
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
import torch as t

path = Path("data_engineered")
train_df = pd.read_csv(path/'train_data.csv')

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    t.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if t.cuda.is_available():
        t.cuda.manual_seed(seed_value)
        t.cuda.manual_seed_all(seed_value)
        t.backends.cudnn.deterministic = True
        t.backends.cudnn.benchmark = False

seed_everything(42)

path = Path("data_engineered")
train_df = pd.read_csv(path/'train_data.csv')
train_df['path'] = train_df['image_id'].map(lambda x:path/'train'/x)
train_df = train_df.drop(columns=['image_id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
    
def get_train_aug(sz): 
    return albumentations.Compose([
            albumentations.CenterCrop(sz,sz, p=1.),

            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.15, 
                sat_shift_limit=0.15, 
                val_shift_limit=0.15, 
                p=0.25
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.25
            ),
            albumentations.CoarseDropout(p=0.1),
            albumentations.Cutout(p=0.1)
])

def get_valid_aug(sz): 
    return albumentations.Compose([
    albumentations.CenterCrop(sz,sz, p=1.),
    albumentations.Resize(sz,sz)
], p=1.)


def get_dls(sz,bs):
    item_tfms = AlbumentationsTransform(get_train_aug(sz), get_valid_aug(sz))
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]
    dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=999, #seed
                                   label_col=0, #label is in the first column of the DataFrame
                                   fn_col=1, #filename/path is in the second column of the DataFrame
                                   bs=bs, #pass in batch size
                                   item_tfms=item_tfms, #pass in item_tfms
                                   batch_tfms=batch_tfms) #pass in batch_tfms
    return dls


dls = get_dls(256,16)
device = torch.device('cuda')

/home/p/.is/anaconda3/lib/python3.9/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [2]:
vgg19 = vgg19_bn(pretrained=True)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x
    
vgg19.avgpool = nn.AdaptiveAvgPool2d((1, 1))
# vgg19.classifier = Identity()
# vgg19.classifier = nn.Linear(512, 2)
vgg19.classifier = nn.Sequential(*[nn.Linear(512, 256), nn.ReLU(), nn.Dropout(.3), nn.Linear(256, 2)])

loss_func = LabelSmoothingCrossEntropyFlat(weight=torch.tensor([2.145765712599587, 1.]).to(device))
learn = Learner(dls, vgg19, opt_func=ranger, loss_func=loss_func, metrics=F1Score())
callbacks = [SaveModelCallback(monitor='valid_loss', fname='vgg19')]

In [3]:
learn.fit_one_cycle(20, wd=1e-4, div=1e5, lr_max=0.0001, pct_start=.1, cbs=callbacks)
learn.fit_one_cycle(10, wd=1e-4, div=1e5, lr_max=0.0001, pct_start=.1, cbs=callbacks)

Better model found at epoch 0 with valid_loss value: 0.3745611608028412.
Better model found at epoch 2 with valid_loss value: 0.298328161239624.
Better model found at epoch 3 with valid_loss value: 0.28833961486816406.
Better model found at epoch 4 with valid_loss value: 0.2736191749572754.
Better model found at epoch 5 with valid_loss value: 0.2667563855648041.
Better model found at epoch 7 with valid_loss value: 0.2662286162376404.
Better model found at epoch 8 with valid_loss value: 0.2626535892486572.
Better model found at epoch 9 with valid_loss value: 0.24925827980041504.
Better model found at epoch 10 with valid_loss value: 0.24377115070819855.
Better model found at epoch 12 with valid_loss value: 0.23916003108024597.
Better model found at epoch 13 with valid_loss value: 0.23490740358829498.
Better model found at epoch 14 with valid_loss value: 0.23002903163433075.
Better model found at epoch 15 with valid_loss value: 0.2271135449409485.
Better model found at epoch 17 with valid

Better model found at epoch 0 with valid_loss value: 0.24846278131008148.
Better model found at epoch 1 with valid_loss value: 0.24274282157421112.
Better model found at epoch 2 with valid_loss value: 0.24257944524288177.
Better model found at epoch 3 with valid_loss value: 0.23911051452159882.
Better model found at epoch 5 with valid_loss value: 0.23003152012825012.
Better model found at epoch 6 with valid_loss value: 0.2272375226020813.
Better model found at epoch 7 with valid_loss value: 0.22286133468151093.
Better model found at epoch 8 with valid_loss value: 0.22046686708927155.
Better model found at epoch 9 with valid_loss value: 0.2193494737148285.


In [4]:
test_df = pd.read_csv(path/'validation_data'/'C-NMC_test_prelim_phase_data_labels.csv')
del test_df['Patient_ID']
test_df.columns = train_df.columns[::-1]
test_df.columns = ['old_path', 'label']
test_df['path'] = test_df['old_path'].map(lambda x:path/'validation_data'/'C-NMC_test_prelim_phase_data'/x)
test_df = test_df.drop(columns=['old_path'])
test_dl = learn.dls.test_dl(test_df)

In [5]:
preds, y_preds = learn.tta(dl=test_dl)
y = np.array(test_df['label'])
np.save('vgg19_fresh_weighted_preds_raw_full.npy', preds.numpy())
recovered = np.load('vgg19_fresh_weighted_preds_raw_full.npy') # load


In [6]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)


tp, fp, tn, fn = perf_measure(y, np.argmax(preds.numpy(), axis=1))
print(tp / (tp + fn), ' sensitivity, aka recall in binary setting') # Sensitivity resnet
print(tp / (tp + fp), ' specificity, precision') # Specificity resnet
print((tp + tn) / (tp + tn + fp + fn), ' accuracy') # Accuracy resnet
print(f1_score(y, np.argmax(preds.numpy(), axis=1), average='weighted'), ' f1score')

0.8408531583264971  sensitivity, aka recall in binary setting
0.8859118409680208  specificity, precision
0.8253883235136583  accuracy
0.8270961216987974  f1score


In [7]:
res_preds = np.load('resnet_fresh_weighted_preds_raw.npy') # load
combined_preds = np.argmax((preds.numpy() + res_preds) / 2, axis=1)
tp, fp, tn, fn = perf_measure(y, combined_preds)
print(tp / (tp + fn), ' sensitivity, aka recall in binary setting') # Sensitivity resnet
print(tp / (tp + fp), ' specificity, precision') # Specificity resnet
print((tp + tn) / (tp + tn + fp + fn), ' accuracy') # Accuracy resnet
print(f1_score(y, combined_preds, average='weighted'), ' f1score')

0.8843314191960624  sensitivity, aka recall in binary setting
0.8700564971751412  specificity, precision
0.838243170862346  accuracy
0.8376335672264648  f1score


In [9]:
hybrid_preds = np.load('hybrid_fresh_weighted_preds_raw.npy') # load
combined_preds = np.argmax((preds.numpy() + hybrid_preds + resnet_preds) / 3, axis=1)
tp, fp, tn, fn = perf_measure(y, combined_preds)
print(tp / (tp + fn), ' sensitivity, aka recall in binary setting') # Sensitivity resnet
print(tp / (tp + fp), ' specificity, precision') # Specificity resnet
print((tp + tn) / (tp + tn + fp + fn), ' accuracy') # Accuracy resnet
print(f1_score(y, combined_preds, average='weighted'), ' f1score')

0.8539786710418376  sensitivity, aka recall in binary setting
0.8905047048759623  specificity, precision
0.8361006963042313  accuracy
0.8374290246912366  f1score
